In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [3]:
df = df[(df['press'] == 'Entrepreneur') | (df['press'] == 'Bangkok Post')]
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
289,"{'id': None, 'name': 'Entrepreneur'}",Yumi Clark,5 Tips for Women Entrepreneurs to Close the Sa...,Women workers make 76 cents on the dollar comp...,https://www.entrepreneur.com/article/303556,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T18:00:00Z,Ask nearly any small business owner and they'l...,Entrepreneur
331,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,The New World of Innovation-as-a-Service,"Capturing innovation from ""the crowd"" (as in c...",https://www.entrepreneur.com/article/304483,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T20:30:00Z,Technological disruption is shattering all tho...,Entrepreneur
460,"{'id': None, 'name': 'Entrepreneur'}",Yoav Vilner,4 Emerging Fintech Trends Relevant to Every En...,Cryptocurrencies are important but there is a ...,https://www.entrepreneur.com/article/303581,https://assets.entrepreneur.com/content/3x2/13...,2017-11-11T16:00:00Z,"Be innovative, or get left behind. 2018 is fas...",Entrepreneur
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post
918,"{'id': None, 'name': 'Entrepreneur'}",The Oracles,The Most Valuable Lessons These 6 Top Entrepre...,Hard-won wisdom that created wildly successful...,https://www.entrepreneur.com/article/304497,https://assets.entrepreneur.com/content/3x2/13...,2017-11-13T16:15:00Z,The entrepreneur’s journey is full of challeng...,Entrepreneur
...,...,...,...,...,...,...,...,...,...
731811,"{'id': None, 'name': 'Bangkok Post'}",Post Reporters,Cabinet works on poll pledges,The government looks set to roll out a number ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-03T23:28:00Z,The government looks set to roll out a number ...,Bangkok Post
735759,"{'id': None, 'name': 'Bangkok Post'}",Wassayos Ngamkham,Three Chinese crypto fraud suspects held,Three Chinese nationals have been arrested in ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-12T13:37:00Z,Three Chinese nationals have been arrested in ...,Bangkok Post
736061,"{'id': None, 'name': 'Bangkok Post'}","Wichit Chantanusornsiri, Somruedi Banchongduan...","Probing the B10,000 digital handout",The Pheu Thai Party-led coalition government b...,https://www.bangkokpost.com/business/general/2...,https://static.bangkokpost.com/media/content/d...,2023-09-12T23:14:00Z,The Pheu Thai Party-led coalition government b...,Bangkok Post
736193,"{'id': None, 'name': 'Bangkok Post'}",Reuters and Online Reporters,US jails OneCoin fraudster for 20 years,NEW YORK: A fake cryptocurrency promoter who w...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-13T07:36:00Z,NEW YORK: A fake cryptocurrency promoter who w...,Bangkok Post


In [4]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(df):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        url = row['url']
        press = row['press']
        
        # press 값에 따른 selector 설정
        if press == 'Entrepreneur':
            selector = '.leading-relaxed'
        elif press == 'Bangkok Post':
            selector = '.article-content'
        else:
            print(f"Unknown press value: {press} for URL: {url}")
            contents.append('')
            continue

        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            article_elements = soup.select(selector)

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [5]:
head = df.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:02<00:00,  1.77it/s]
<ipython-input-5-6286cb6d68db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
289,"{'id': None, 'name': 'Entrepreneur'}",Yumi Clark,5 Tips for Women Entrepreneurs to Close the Sa...,Women workers make 76 cents on the dollar comp...,https://www.entrepreneur.com/article/303556,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T18:00:00Z,Ask nearly any small business owner and they'l...,Entrepreneur,\nAsk nearly any small business owner and they...
331,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,The New World of Innovation-as-a-Service,"Capturing innovation from ""the crowd"" (as in c...",https://www.entrepreneur.com/article/304483,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T20:30:00Z,Technological disruption is shattering all tho...,Entrepreneur,\nTechnological disruption is shattering all t...
460,"{'id': None, 'name': 'Entrepreneur'}",Yoav Vilner,4 Emerging Fintech Trends Relevant to Every En...,Cryptocurrencies are important but there is a ...,https://www.entrepreneur.com/article/303581,https://assets.entrepreneur.com/content/3x2/13...,2017-11-11T16:00:00Z,"Be innovative, or get left behind. 2018 is fas...",Entrepreneur,"\nBe innovative, or get left behind. 2018 is f..."
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post,\n\n\n\n\n Even elite...
918,"{'id': None, 'name': 'Entrepreneur'}",The Oracles,The Most Valuable Lessons These 6 Top Entrepre...,Hard-won wisdom that created wildly successful...,https://www.entrepreneur.com/article/304497,https://assets.entrepreneur.com/content/3x2/13...,2017-11-13T16:15:00Z,The entrepreneur’s journey is full of challeng...,Entrepreneur,\nThe entrepreneur's journey is full of challe...


In [6]:
# Adding the content to the DataFrame
df['content_full'] = get_contents(df)
df

100%|██████████| 2476/2476 [43:43<00:00,  1.06s/it] 
<ipython-input-6-0b636d97f36d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_full'] = get_contents(df)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
289,"{'id': None, 'name': 'Entrepreneur'}",Yumi Clark,5 Tips for Women Entrepreneurs to Close the Sa...,Women workers make 76 cents on the dollar comp...,https://www.entrepreneur.com/article/303556,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T18:00:00Z,Ask nearly any small business owner and they'l...,Entrepreneur,\nAsk nearly any small business owner and they...
331,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,The New World of Innovation-as-a-Service,"Capturing innovation from ""the crowd"" (as in c...",https://www.entrepreneur.com/article/304483,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T20:30:00Z,Technological disruption is shattering all tho...,Entrepreneur,\nTechnological disruption is shattering all t...
460,"{'id': None, 'name': 'Entrepreneur'}",Yoav Vilner,4 Emerging Fintech Trends Relevant to Every En...,Cryptocurrencies are important but there is a ...,https://www.entrepreneur.com/article/303581,https://assets.entrepreneur.com/content/3x2/13...,2017-11-11T16:00:00Z,"Be innovative, or get left behind. 2018 is fas...",Entrepreneur,"\nBe innovative, or get left behind. 2018 is f..."
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post,\n\n\n\n\n Even elite...
918,"{'id': None, 'name': 'Entrepreneur'}",The Oracles,The Most Valuable Lessons These 6 Top Entrepre...,Hard-won wisdom that created wildly successful...,https://www.entrepreneur.com/article/304497,https://assets.entrepreneur.com/content/3x2/13...,2017-11-13T16:15:00Z,The entrepreneur’s journey is full of challeng...,Entrepreneur,\nThe entrepreneur's journey is full of challe...
...,...,...,...,...,...,...,...,...,...,...
731811,"{'id': None, 'name': 'Bangkok Post'}",Post Reporters,Cabinet works on poll pledges,The government looks set to roll out a number ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-03T23:28:00Z,The government looks set to roll out a number ...,Bangkok Post,\n\n\n\n\n Prime Mini...
735759,"{'id': None, 'name': 'Bangkok Post'}",Wassayos Ngamkham,Three Chinese crypto fraud suspects held,Three Chinese nationals have been arrested in ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-12T13:37:00Z,Three Chinese nationals have been arrested in ...,Bangkok Post,\n\n\n\n\n Pol Lt Gen...
736061,"{'id': None, 'name': 'Bangkok Post'}","Wichit Chantanusornsiri, Somruedi Banchongduan...","Probing the B10,000 digital handout",The Pheu Thai Party-led coalition government b...,https://www.bangkokpost.com/business/general/2...,https://static.bangkokpost.com/media/content/d...,2023-09-12T23:14:00Z,The Pheu Thai Party-led coalition government b...,Bangkok Post,\n\n\n\n\n The Pheu T...
736193,"{'id': None, 'name': 'Bangkok Post'}",Reuters and Online Reporters,US jails OneCoin fraudster for 20 years,NEW YORK: A fake cryptocurrency promoter who w...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-13T07:36:00Z,NEW YORK: A fake cryptocurrency promoter who w...,Bangkok Post,\n\n\n\n\n The OneCoi...


In [285]:
head['url'].iloc[3]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [296]:
head['content_full'].iloc[0]

"\n\n\n\n\n\nLipin Pappachan, Tech Times\n                    \n                21 November 2017, 09:11 am\n\n\n\nNearly $31 million worth of bitcoin currency was wiped off from cryptocurrency\xa0startup Tethers' Treasury account. The startup is in business with bitcoin exchanges and the hacker was able to move the bitcoin to an unauthorized account.\nTether has released the details of the attack and precautionary measures on its website for its users.\nDetails Of The Attack That Stole $31 Million\nOn Nov. 19, 2017, the Treasury account of Tether was compromised by a hacker. The bitcoins were maliciously transferred from Tether to the following address: 16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r by the hacker. Although the fund address has now been flagged, the curtailment of further damage will depend on how the startup is able to control the spread of this fund in the broader ecosystem. Tether has requested people who trade in bitcoins to not to purchase USDT tokens from the above address.\n

In [7]:
empty_rows = df[df['content_full'] == '']
print(len(empty_rows))

330


In [10]:
empty_rows['url'].iloc[200]

'https://www.entrepreneur.com/article/373400'

<!-- # empty_rows['url'].iloc[0] -->

In [11]:
Entrepreneur = df[df['press'] == 'Entrepreneur']
Entrepreneur

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
289,"{'id': None, 'name': 'Entrepreneur'}",Yumi Clark,5 Tips for Women Entrepreneurs to Close the Sa...,Women workers make 76 cents on the dollar comp...,https://www.entrepreneur.com/article/303556,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T18:00:00Z,Ask nearly any small business owner and they'l...,Entrepreneur,\nAsk nearly any small business owner and they...
331,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,The New World of Innovation-as-a-Service,"Capturing innovation from ""the crowd"" (as in c...",https://www.entrepreneur.com/article/304483,https://assets.entrepreneur.com/content/3x2/13...,2017-11-10T20:30:00Z,Technological disruption is shattering all tho...,Entrepreneur,\nTechnological disruption is shattering all t...
460,"{'id': None, 'name': 'Entrepreneur'}",Yoav Vilner,4 Emerging Fintech Trends Relevant to Every En...,Cryptocurrencies are important but there is a ...,https://www.entrepreneur.com/article/303581,https://assets.entrepreneur.com/content/3x2/13...,2017-11-11T16:00:00Z,"Be innovative, or get left behind. 2018 is fas...",Entrepreneur,"\nBe innovative, or get left behind. 2018 is f..."
918,"{'id': None, 'name': 'Entrepreneur'}",The Oracles,The Most Valuable Lessons These 6 Top Entrepre...,Hard-won wisdom that created wildly successful...,https://www.entrepreneur.com/article/304497,https://assets.entrepreneur.com/content/3x2/13...,2017-11-13T16:15:00Z,The entrepreneur’s journey is full of challeng...,Entrepreneur,\nThe entrepreneur's journey is full of challe...
1756,"{'id': None, 'name': 'Entrepreneur'}",Andrew Medal,How Blockchain Is Creating a New Future for Di...,The transparent nature of blockchain data will...,https://www.entrepreneur.com/article/304654,https://assets.entrepreneur.com/content/3x2/13...,2017-11-15T21:15:00Z,Advertising today is a multi-billion-dollar in...,Entrepreneur,\nAdvertising today is a multi-billion-dollar ...
...,...,...,...,...,...,...,...,...,...,...
624467,"{'id': None, 'name': 'Entrepreneur'}",Gleb Tsipursky,26% of U.S. Workers Would Rather Undergo a Roo...,This staggering statistic reveals a clear shif...,https://www.entrepreneur.com/growth-strategies...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-27T23:00:00Z,According to a recent survey conducted by job ...,Entrepreneur,\nAccording to a recent survey conducted by jo...
625490,"{'id': None, 'name': 'Entrepreneur'}",Steve Taplin,Why Software Talent Is Still in Demand Despite...,Despite the layoffs and fears of an economic r...,https://www.entrepreneur.com/science-technolog...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:30:00Z,Talk of an economic downturn and potential rec...,Entrepreneur,\nTalk of an economic downturn and potential r...
625492,"{'id': None, 'name': 'Entrepreneur'}",Sam Silverman,The FBI's Most Wanted 'Crypto Queen' Scammed I...,Ruja Ignatova vanished in 2017 after orchestra...,https://www.entrepreneur.com/business-news/who...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T19:36:00Z,"Disgraced ""Crypto Queen"" Ruja Ignatova'stime o...",Entrepreneur,"\nDisgraced ""Crypto Queen"" Ruja Ignatova's tim..."
625540,"{'id': None, 'name': 'Entrepreneur'}",Evgeniya Zaslavskaya,How to Build a Reputation That Will Become a R...,Here are a few tips on building a personal bra...,https://www.entrepreneur.com/growing-a-busines...,https://assets.entrepreneur.com/content/3x2/20...,2023-01-30T21:00:00Z,"As Robert Greene wrote in his book, The 48 Law...",Entrepreneur,"\nAs Robert Greene wrote in his book, The 48 L..."


In [12]:
BangkokPost = df[df['press'] == 'Bangkok Post']
BangkokPost

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
614,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Foreign gangs in government crosshairs,"Almost 100,000 foreign nationals who have over...",https://www.bangkokpost.com/news/general/13594...,https://www.bangkokpost.com/media/content/2017...,2017-11-12T21:30:00Z,Even elite units such as the Special Weapons A...,Bangkok Post,\n\n\n\n\n Even elite...
1068,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Think bitcoin's getting expensive? Try Zimbabwe,With Zimbabwe under threat of cash shortages a...,https://www.bangkokpost.com/vdo/business-and-p...,https://www.bangkokpost.com/media/content/2017...,2017-11-14T01:17:00Z,Readers are urged not to submit comments that ...,Bangkok Post,
2276,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Bitcoin emerges as crisis currency,NEW YORK: About a third of the customers queui...,https://www.bangkokpost.com/business/world/136...,https://www.bangkokpost.com/media/content/2017...,2017-11-17T08:51:00Z,A bitcoin coin is seen in an illustration pict...,Bangkok Post,\n\n\n\n\n A bitcoin ...
2646,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,My beloved dog must suffer,My beloved dog of 15 years is dying. His desce...,https://www.bangkokpost.com/lifestyle/social-a...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-18T21:00:00Z,PHOTO: 123RF My beloved dog of 15 years is dyi...,Bangkok Post,\nMy beloved dog of 15 years is dying. His des...
2809,"{'id': None, 'name': 'Bangkok Post'}",Bangkok Post Public Company Limited,Logistics 4.0,The fourth industrial revolution and rapid ado...,https://www.bangkokpost.com/business/news/1363...,https://www.bangkokpost.com/media/content/dcx/...,2017-11-20T05:30:00Z,The fourth industrial revolution and rapid ado...,Bangkok Post,\nThe fourth industrial revolution and rapid a...
...,...,...,...,...,...,...,...,...,...,...
731811,"{'id': None, 'name': 'Bangkok Post'}",Post Reporters,Cabinet works on poll pledges,The government looks set to roll out a number ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-03T23:28:00Z,The government looks set to roll out a number ...,Bangkok Post,\n\n\n\n\n Prime Mini...
735759,"{'id': None, 'name': 'Bangkok Post'}",Wassayos Ngamkham,Three Chinese crypto fraud suspects held,Three Chinese nationals have been arrested in ...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-12T13:37:00Z,Three Chinese nationals have been arrested in ...,Bangkok Post,\n\n\n\n\n Pol Lt Gen...
736061,"{'id': None, 'name': 'Bangkok Post'}","Wichit Chantanusornsiri, Somruedi Banchongduan...","Probing the B10,000 digital handout",The Pheu Thai Party-led coalition government b...,https://www.bangkokpost.com/business/general/2...,https://static.bangkokpost.com/media/content/d...,2023-09-12T23:14:00Z,The Pheu Thai Party-led coalition government b...,Bangkok Post,\n\n\n\n\n The Pheu T...
736193,"{'id': None, 'name': 'Bangkok Post'}",Reuters and Online Reporters,US jails OneCoin fraudster for 20 years,NEW YORK: A fake cryptocurrency promoter who w...,https://www.bangkokpost.com/thailand/general/2...,https://static.bangkokpost.com/media/content/2...,2023-09-13T07:36:00Z,NEW YORK: A fake cryptocurrency promoter who w...,Bangkok Post,\n\n\n\n\n The OneCoi...


In [15]:
Entrepreneur.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\Entrepreneur_83.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content